In [20]:
import pandas as pd
import numpy as np
import json
from pycocotools.coco import COCO
from skimage import io

In [5]:
train = COCO('/media/saket/014178da-fdf2-462c-b901-d5f4dbce2e275/tiger/det/tiger_train.json')   

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


In [12]:
f=open('train.txt','w')
ids=train.getImgIds()
id_to_fname={k:train.imgs[k]['file_name'] for k in train.imgs.keys()} 
ww=1920
hh=1080

In [14]:
for id in ids: 
    data=[] 
    a=train.getAnnIds(id) 
    filename='labels/'+id_to_fname[id].replace('.jpg','.txt') 
    print('data/tiger/images/'+id_to_fname[id],file=f) 
    for item in a: 
        cls=[0] 
        bbox=train.anns[item]['bbox'] 
        x1,y1,w,h=bbox 
        x2,y2=x1+w,y1+h
        cx,cy=(x1+x2)/2,(y1+y2)/2 
        cx,cy,w,h=cx/ww,cy/hh,w/ww,h/hh 
        cls.extend([cx,cy,w,h]) 
        data.append(cls) 
        df = pd.DataFrame(data) 
        df.to_csv(filename,header=None,index=False,sep=' ')                  


In [15]:
f.close()

In [16]:
val = COCO('/media/saket/014178da-fdf2-462c-b901-d5f4dbce2e275/tiger/det/tiger_val.json')   
f=open('val.txt','w')
ids=val.getImgIds()
id_to_fname={k:val.imgs[k]['file_name'] for k in val.imgs.keys()} 
ww=1920
hh=1080
for id in ids: 
    data=[] 
    a=val.getAnnIds(id) 
    filename='labels/'+id_to_fname[id].replace('.jpg','.txt') 
    print('data/tiger/images/'+id_to_fname[id],file=f) 
    for item in a: 
        cls=[0] 
        bbox=val.anns[item]['bbox'] 
        x1,y1,w,h=bbox 
        x2,y2=x1+w,y1+h
        cx,cy=(x1+x2)/2,(y1+y2)/2 
        cx,cy,w,h=cx/ww,cy/hh,w/ww,h/hh 
        cls.extend([cx,cy,w,h]) 
        data.append(cls) 
        df = pd.DataFrame(data) 
        df.to_csv(filename,header=None,index=False,sep=' ')                  
f.close()

loading annotations into memory...
Done (t=0.30s)
creating index...
index created!


In [18]:
import torch

In [56]:
boxes = torch.from_numpy(np.loadtxt('./labels/0002.txt').reshape(-1, 5))
boxes

tensor([[0.0000, 0.2638, 0.4657, 0.3286, 0.3648]], dtype=torch.float64)

In [37]:
img=io.imread('images/0002.jpg')
import sys
sys.path.insert(0,'../../utils/')
import utils
import torch.nn.functional as F
def pad_to_square(img, pad_value):
    c, h, w = img.shape
    dim_diff = np.abs(h - w)
    # (upper / left) padding and (lower / right) padding
    pad1, pad2 = dim_diff // 2, dim_diff - dim_diff // 2
    # Determine padding
    pad = (0, 0, pad1, pad2) if h <= w else (pad1, pad2, 0, 0)
    # Add padding
    img = F.pad(img, pad, "constant", value=pad_value)

    return img, pad


In [48]:
img  = torch.FloatTensor(np.transpose(img,(2,0,1)))
_, h, w = img.shape
h_factor, w_factor = (h, w) 
# Pad to square resolution
img, pad = pad_to_square(img, 0)
_, padded_h, padded_w = img.shape

# ---------
#  Label
# ---------


In [49]:
h_factor, w_factor

(1080, 1920)

In [58]:
# Extract coordinates for unpadded + unscaled image
x1 = w_factor * (boxes[:, 1] - boxes[:, 3] / 2)
y1 = h_factor * (boxes[:, 2] - boxes[:, 4] / 2)
x2 = w_factor * (boxes[:, 1] + boxes[:, 3] / 2)
y2 = h_factor * (boxes[:, 2] + boxes[:, 4] / 2)
# Adjust for added padding
x1 += pad[0]
y1 += pad[2]
x2 += pad[1]
y2 += pad[3]
# Returns (x, y, w, h)
boxes[:, 1] = ((x1 + x2) / 2) / padded_w
boxes[:, 2] = ((y1 + y2) / 2) / padded_h
boxes[:, 3] *= w_factor / padded_w
boxes[:, 4] *= h_factor / padded_h

targets = torch.zeros((len(boxes), 6))
targets[:, 1:] = boxes

In [59]:
targets

tensor([[0.0000, 0.0000, 0.2638, 0.4807, 0.3286, 0.2052]])

In [52]:
boxes

tensor([[0.0000, 0.2638, 0.4807, 0.3286, 0.2052]], dtype=torch.float64)

In [53]:
pad


(0, 0, 420, 420)

In [55]:
1080+420+420

1920

In [57]:
boxes

tensor([[0.0000, 0.2638, 0.4657, 0.3286, 0.3648]], dtype=torch.float64)